<a href="https://colab.research.google.com/github/eunnjji/MultiCampus-EmpAcademy/blob/main/project2/4%EC%A1%B0_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4조

- 팀명: SYH : super youth hero
- 주제: 기계 시설물 분야 AI 학습용 데이터를 활용한 기계 시설물의 고장 여부 예측 모델 개발
- 여수진,성은지,홍승표,문지수 
- [참고](https://www.aidatahackathon.com/)

## 절차
1. 문제 분석
1. 데이터 정제
1. 모델 생성 및 평가
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# libarary import
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn as skl
import pickle

# seed setting
np.random.seed(42)
tf.random.set_seed(42)

BASE_DIR = "/content/drive/MyDrive/"
DATA_DIR = '/content/drive/MyDrive/project/daejeon_data'

## 데이터 형태
csv 파일  
,(콤마)로 분류된 시계열 데이터  
+ current  
  - 파일명, 에러 아이디, RPM, 정격전류,
  - 사용전류, Sample Rate 값, R전류의 RMS (RRMS),
  - R 전류의 MEAN (RMEAN), R 전류의 MAX (RMAX),
  - S 전류의 RMS (SRMS), S 전류의 MEAN (SMEAN),
  - S 전류의 MAX (SMAX), T 전류의 RMS (TRMS),
  - T 전류의 MEAN (TMEAN), T 전류의 MAX (TMAX)
+ kimm
  - 파일명, Sample Rate 값, Label, Label_No
+ vibration
  - 파일명, Sample Rate, Sample Rate, Sample Rate

## 데이터 구조

op-csv : 모터 정상 가동 중 측정한 데이터  
no-csv : 모터가 해당 고장 레이블인 상태에 측정한 데이터

current(전류 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
kimm(유선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
vibration(무선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  

|레이블|
|---|
|정상|
|회전체 불평형(unbalance)|
|베어링 불량(bearing)|
|벨트 느슨함(belt)|
|축정렬 불량(misalignment)|
<br>

전류의 경우 \[r,s,t value, kW] target(y)는 \[레이블종류] 특성 \[kw]  
유선진동의 경우 \[Sample rate value, kW] target(y)는 \[레이블종류] 특성 \[kw]  
무선진동의 경우 \[Sample rate value, kW] target(y)는 \[레이블종류] 특성 \[kw]  


# 1. 문제 분석

지도 학습 문제, 회귀 문제,   
다중 분류 (소프트맥스 회귀)  
CNN 계열 모델 활용 


# 2. 데이터 정제 및 분석


,로 구분된 csv 파일 형태로 데이터 가져오기

In [ ]:
%cd '/content/drive/MyDrive'
os.getcwd()

/content/drive/MyDrive


'/content/drive/MyDrive'

In [ ]:
classSet = ("bearing", "belt", "misalignment", "unbalance")

def DatadirLoad(dirname):
    csv_op_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}
    csv_no_dict = {classSet[0]:[], classSet[1]:[], classSet[2]:[],classSet[3]:[]}

    Save_op_Name, Save_no_Name = 'Data_op_dir.pickle','Data_no_dir.pickle'
    Save_op_path = os.path.join(BASE_DIR,Save_op_Name)
    Save_no_path = os.path.join(BASE_DIR,Save_no_Name)

    if os.path.isdir(Save_op_path):
        with open(Save_op_path, 'rb') as fr:
            csv_op_dict = picle.load(fr)
    if os.path.isdir(Save_no_path):
        with open(Save_no_path, 'rb') as fr:
            csv_no_dict = picle.load(fr)
        return csv_op_dict,csv_no_dict

    dirpath = os.path.join(DATA_DIR,dirname)
    if os.path.isdir(dirpath):
        for d in os.listdir(dirpath):
            d_path = dirpath+'/'+d
            if os.path.isdir(d_path):
                Error_Class_Lable,kw_Name,*_ = d.split('_')
                print(Error_Class_Lable,kw_Name)
                data_dir_list = [ x for x in os.listdir(d_path) if len(x)<2]
                data_dir_list.sort()
                #print(data_dir_list)
                if len(data_dir_list) > 0:
                    for d2 in data_dir_list:
                        d2_path = os.path.join(d_path,d2)

                        # op-csv file # 정상 작동
                        d2_op_path = os.path.join(d2_path,'4.op-csv')
                        d2_op_data = [x for x in os.listdir(d2_op_path) if x.endswith('.csv')]
                        op_csv_path = os.path.join(d2_op_path,d2_op_data[0])
                        csv_op_dict[Error_Class_Lable].append(op_csv_path)
                        with open(Save_op_path,'wb') as fw:
                            pickle.dump(csv_op_dict,fw)
                        # no-csv file # 고장 발생
                        d2_no_path = os.path.join(d2_path,'5.no-csv')
                        d2_no_data = [x for x in os.listdir(d2_no_path) if x.endswith('.csv')]
                        no_csv_path = os.path.join(d2_no_path,d2_no_data[0])
                        csv_no_dict[Error_Class_Lable].append(no_csv_path)
                        with open(Save_no_path,'wb') as fw:
                            pickle.dump(csv_no_dict,fw)

    return csv_op_dict, csv_no_dict

범주형 데이터 -> 숫자형 데이터 변환하기 위한 사전 단계

In [ ]:
class_name = {'bearing':0, 'belt':1, 'misalignment':2, 'normal':3, 'unbalance':4}

In [ ]:
def read_csv_dir(path, max_row_size):
    t_pd = None
    update_row_size = max_row_size
    y = None
    if os.path.isfile(path):
        data_Name = os.path.basename(path)
        dir_name,Ekw,dir_num,opno  = os.path.dirname(path).split('/')[6:]
        Error_Class_Lable,kw_Name,*_ = Ekw.split('_')      
        opno = 0 if opno[0] == '4' else 1
        # 정상 : False(0), 고장 : True(1)

        if dir_name[0] == 'c': #current(전류)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(15))
            t_pd.drop(3, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)

        elif dir_name[0] == 'k': # kimm(유선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)

        elif dir_name[0] == 'v': #vibration(무선진동)
            t_pd = pd.read_csv(path, header=None,encoding='utf-8-sig',skiprows=range(4))
            t_pd.drop(1, axis=1, inplace=True)
            t_pd['Motor'] = float(kw_Name[:-2])
            t_pd['target'] = class_name[Error_Class_Lable] if opno == 1 else class_name['normal']
            y = t_pd['target']
            t_pd.drop(['target'], axis=1, inplace=True)
            if (t_pd.values.shape[0]) > max_row_size:
                update_row_size = t_pd.values.shape[0]
            elif (t_pd.values.shape[0]) < max_row_size:
                update_row_size = 6000
                tmp = np.zeros((int(update_row_size - t_pd.values.shape[0]),int(t_pd.values.shape[1])))
                #y_tmp = np.zeros(y.values.shape) # 이렇게 될 경우 normal 데이터에 bearing 데이터가 함께 존재하게 됨
                y_tmp = y.values
                new_t = np.vstack([t_pd.values,tmp])
                new_y = np.append(y,y_tmp,axis=0)
                return new_t.tolist(), update_row_size, new_y.tolist()

        else: 
            print('error')
            print('dir_category : {}, lable : {}, op: {}'.format(dir_name, Error_Class_Lable,opno))
            return 0
    return t_pd.values.tolist(), update_row_size, y.values.tolist()
    

DataSet 생성

In [ ]:
# dataset 생성
def makeDataset(op_dict, no_dict, category_num, shape_op = 0):
    """
    input: 정상data의 path list, 오류 data의 path list
    output: numpy의 ndarray 타입 형태로 dataset 반환
    category_num:
    0은 current 데이터
    1은 유선 진동 데이터 
    2는 무선 진동 데이터 
    shape_op :
     0이면 (38200, 60 , 5(특성 수)) 형태, 
     1이면 (38200(data개수 * row/60한 값), 5(특성 수), 60)
    """
    tmp = []
    max_row_size = 0
    y_tmp = []
    Y = None

    category_name = None
    if category_num == 0:
        category_name = 'current'
    elif category_num == 1:
        category_name = 'kimm'
    else:category_name = 'vibration'

    save_x_Name, save_y_Name = category_name+'_x.pickle',category_name+'_y.pickle'
    save_x_path = os.path.join(BASE_DIR,save_x_Name)
    save_y_path = os.path.join(BASE_DIR,save_y_Name)



    for k,v in op_dict.items():
        for p in v:
            data, max, y = read_csv_dir(p, max_row_size)
            tmp.append(data)
            y_tmp.append(y)
            max_row_size = max
    for k,v in no_dict.items():
        for p in v:
            data, max, y = read_csv_dir(p, max_row_size)
            tmp.append(data)
            y_tmp.append(y)
            max_row_size = max
    
    if shape_op == 0:
        dataset = np.stack(tmp)
        feature_num = 4 if category_num == 0 else 2
        Y = np.stack(y_tmp)
        shape = (-1,60,feature_num) 
        y_shape = (-1,60)

    else:
        dataset = np.stack(tmp,axis=2)
        feature_num = 4 if category_num == 0 else 2
        dataset = dataset.reshape((-1,feature_num,60)).copy()
        Y = np.stack(y_tmp)
        Y = Y.reshape((-1,60)).copy()
        shape = (-1,feature_num,60)
        y_shape = (-1,60)
    with open(save_x_path,'wb') as fw:
        pickle.dump(dataset.reshape(shape),fw)
    with open(save_y_path,'wb') as fw:
        pickle.dump(Y.reshape(y_shape),fw)

    return  dataset.reshape(shape), Y.reshape(y_shape)

생성된 DataSet 저장 및 불러오기

In [ ]:
def load_dataset(category_num=0):
    """
    0: current dataset
    1: kimm dataset
    2: vibration dataset
    """
    category_name = None
    if category_num == 0:
        category_name = 'current'
    elif category_num == 1:
        category_name = 'kimm'
    else:category_name = 'vibration'

    load_x_Name, load_y_Name = category_name+'_x.pickle',category_name+'_y.pickle'
    load_x_path = os.path.join(BASE_DIR,load_x_Name) #ex. /content/drive/MyDrive/current_x.pickle
    load_y_path = os.path.join(BASE_DIR,load_y_Name) 

    if os.path.isfile(load_x_path):
        print('exist')
        with open(load_x_path, 'rb') as fr:
            X = pickle.load(fr)
        if os.path.isfile(load_y_path):
            print('exist')
            with open(load_y_path, 'rb') as fr:
                y = pickle.load(fr)
            return X,y
    else:
        print('not exist')
        if category_num == 0:
            current_op_csv,current_no_csv = DatadirLoad("current(전류)")
            current_x , current_y = makeDataset(current_op_csv,current_no_csv,0,shape_op=0)
            return current_x,current_y
        elif category_num == 1:
            kimm_op_csv, kimm_no_csv = DatadirLoad("kimm(유선진동)")
            kimm_x, kimm_y = makeDataset(kimm_op_csv,kimm_no_csv, 1)
            return kimm_x,kimm_y
        else:
            vibration_op_csv, vibration_no_csv = DatadirLoad("vibration(무선진동)")
            vibration_x, vibration_y = makeDataset(vibration_op_csv, vibration_no_csv,2)
            return vibration_x, vibration_y


In [ ]:
pred_name = ['bearing', 'belt', 'misalignment','normal', 'unbalance']

전류 데이터 가져오기

In [ ]:
current_x,current_y = load_dataset(0) # current

exist
exist


In [ ]:
current_x.shape,current_y.shape

((38200, 60, 4), (38200, 60))

유선진동 데이터 가져오기

In [ ]:
kimm_x,kimm_y = load_dataset(1) # kimm

exist
exist


In [ ]:
kimm_x.shape, kimm_y.shape

((76800, 60, 2), (76800, 60))

무선진동 데이터 가져오기

In [ ]:
vibration_x, vibration_y = load_dataset(2) # vibr

exist
exist


In [ ]:
vibration_x.shape, vibration_y.shape

((38400, 60, 2), (38400, 60))

# 3. 모델 생성 및 평가

## 전류 데이터

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train.shape, y_train.shape

((22920, 60, 4), (22920,))

### 일반적인 다층 퍼셉트론(MLP)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[60, 4]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(5, activation="softmax")) 

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               72300     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 102,905
Trainable params: 102,905
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=30,validation_data=(X_valid,y_valid))

Epoch 1/30
717/717 [==============================] - 2s 3ms/step - loss: 1.4015 - accuracy: 0.6331 - val_loss: 0.7161 - val_accuracy: 0.6686
Epoch 2/30
717/717 [==============================] - 2s 2ms/step - loss: 0.7137 - accuracy: 0.6597 - val_loss: 0.7014 - val_accuracy: 0.6449
Epoch 3/30
717/717 [==============================] - 2s 2ms/step - loss: 0.7089 - accuracy: 0.6605 - val_loss: 0.6881 - val_accuracy: 0.6873
Epoch 4/30
717/717 [==============================] - 2s 2ms/step - loss: 0.7068 - accuracy: 0.6636 - val_loss: 0.6811 - val_accuracy: 0.6565
Epoch 5/30
717/717 [==============================] - 2s 2ms/step - loss: 0.6844 - accuracy: 0.6797 - val_loss: 0.7577 - val_accuracy: 0.6571
Epoch 6/30
717/717 [==============================] - 2s 2ms/step - loss: 0.6844 - accuracy: 0.6923 - val_loss: 0.8460 - val_accuracy: 0.6391
Epoch 7/30
717/717 [==============================] - 2s 2ms/step - loss: 0.6587 - accuracy: 0.7167 - val_loss: 0.7386 - val_accuracy: 0.6700
Epoch 

처음 수행시켰을 때 정확도가 10-30을 웃돌았으나  
epoch 반복 수행으로 정확도가 79~90을 웃도는 결과가 나옴
> 30 에폭: 2s 2ms/step - loss: 0.5022 - accuracy: 0.7999 - val_loss: 0.3963 - val_accuracy: 0.8348  
> 30 에폭: 2s 2ms/step - loss: 0.4621 - accuracy: 0.8252 - val_loss: 0.4011 - val_accuracy: 0.8349 (21.02.01)

In [ ]:
X_new = X_train[:5]
y_proba = model.predict(X_new)
y_proba.round(2)

array([[0.23, 0.21, 0.35, 0.04, 0.18],
       [0.2 , 0.21, 0.41, 0.06, 0.12],
       [0.28, 0.17, 0.36, 0.07, 0.12],
       [0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  ]], dtype=float32)

In [ ]:
y_pred = model.predict_classes(X_new)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([2, 2, 2, 3, 3])

In [ ]:
y_train[:5]

array([4, 2, 0, 3, 3])

### CNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(22920, 60, 4, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,3:] = X_train[:,:,3:]/100
X_valid[:,:,3:] = X_valid[:,:,3:]/100
X_test[:,:,3:] = X_test[:,:,3:]/100

In [ ]:
from functools import partial

cnn_model = keras.models.Sequential([
    keras.layers.Conv2D(64,7,activation="relu",padding="same",input_shape=(60,4,1)),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=5, activation='softmax')
])

In [ ]:
cnn_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_144 (Conv2D)          (None, 60, 4, 64)         3200      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 4, 64)         0         
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 60, 4, 128)        73856     
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 60, 4, 128)        147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 2, 128)        0         
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 30, 2, 256)        295168    
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 30, 2, 256)       

In [ ]:
cnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
history = cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
717/717 [==============================] - 9s 10ms/step - loss: 0.6555 - accuracy: 0.6999 - val_loss: 0.0144 - val_accuracy: 0.9973
Epoch 2/10
717/717 [==============================] - 7s 9ms/step - loss: 0.0697 - accuracy: 0.9857 - val_loss: 0.0090 - val_accuracy: 0.9970
Epoch 3/10
717/717 [==============================] - 7s 9ms/step - loss: 0.0329 - accuracy: 0.9932 - val_loss: 0.0348 - val_accuracy: 0.9863
Epoch 4/10
717/717 [==============================] - 7s 9ms/step - loss: 0.0644 - accuracy: 0.9894 - val_loss: 0.0095 - val_accuracy: 0.9973
Epoch 5/10
717/717 [==============================] - 7s 9ms/step - loss: 0.0371 - accuracy: 0.9918 - val_loss: 0.0107 - val_accuracy: 0.9973
Epoch 6/10
717/717 [==============================] - 7s 9ms/step - loss: 0.2544 - accuracy: 0.9936 - val_loss: 0.0091 - val_accuracy: 0.9973
Epoch 7/10
717/717 [==============================] - 7s 9ms/step - loss: 0.0251 - accuracy: 0.9928 - val_loss: 0.0077 - val_accuracy: 0.9973
Epoch

이전 모델보다 훈련, 검증 세트 모두 높은 정확도를 보임
> 10 에폭 : 6s 9ms/step - loss: 0.2158 - accuracy: 0.9838 - val_loss: 0.0566 - val_accuracy: 0.9810  
> 10 에폭 : 6s 9ms/step - loss: 0.0123 - accuracy: 0.9961 - val_loss: 0.0084 - val_accuracy: 0.9971 (21.02.01)

In [ ]:
# score = model.evaluate(X_test, y_test)
X_new = X_train[:10] # pretend we have new images
y_pred = cnn_model.predict(X_new)

In [ ]:
y_pred = cnn_model.predict_classes(X_new)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([4, 2, 0, 3, 3, 3, 3, 4, 1, 4])

In [ ]:
y_train[:10]

array([4, 2, 0, 3, 3, 3, 3, 4, 1, 4])

### ResNet 34

그냥 CNN도 잘 되지만 kimm 데이터에서 CNN을 적용했을 때보다 

ResNet 34를 적용했을 때 더 높은 정확도를 보였기 때문에 전류데이터에도 적용함.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(current_x,current_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train[:,:,3:] = X_train[:,:,3:]/100
X_valid[:,:,3:] = X_valid[:,:,3:]/100
X_test[:,:,3:] = X_test[:,:,3:]/100

In [ ]:
X_train.shape

(22920, 60, 4, 1)

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
current_resnet_model = keras.models.Sequential()
current_resnet_model.add(keras.layers.Conv2D(64,7,strides=1,input_shape=[60,4,1],padding="same",use_bias=False)) # current의 경우 strides를 1로 해야 함
current_resnet_model.add(keras.layers.BatchNormalization())
current_resnet_model.add(keras.layers.Activation("relu"))
current_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    current_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
current_resnet_model.add(keras.layers.GlobalAvgPool2D())
current_resnet_model.add(keras.layers.Flatten())
current_resnet_model.add(keras.layers.Dense(5,activation="softmax"))

In [ ]:
current_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
current_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
717/717 [==============================] - 48s 57ms/step - loss: 0.8097 - accuracy: 0.6336 - val_loss: 1.9073 - val_accuracy: 0.6234
Epoch 2/10
717/717 [==============================] - 40s 56ms/step - loss: 0.5706 - accuracy: 0.7264 - val_loss: 16.8571 - val_accuracy: 0.5012
Epoch 3/10
717/717 [==============================] - 40s 55ms/step - loss: 0.2939 - accuracy: 0.8810 - val_loss: 6.8687 - val_accuracy: 0.6403
Epoch 4/10
717/717 [==============================] - 40s 56ms/step - loss: 0.2336 - accuracy: 0.9157 - val_loss: 5.5375 - val_accuracy: 0.5024
Epoch 5/10
717/717 [==============================] - 40s 56ms/step - loss: 0.1865 - accuracy: 0.9333 - val_loss: 188.2429 - val_accuracy: 0.5898
Epoch 6/10
717/717 [==============================] - 40s 56ms/step - loss: 0.1651 - accuracy: 0.9447 - val_loss: 27.1953 - val_accuracy: 0.5021
Epoch 7/10
717/717 [==============================] - 40s 56ms/step - loss: 0.1783 - accuracy: 0.9409 - val_loss: 3.2406 - val_accur

그냥 CNN을 사용했을 때 보다 훈련, 검증 set의 정확도가 낮게 나왔고 훈련 세트보다 검증세트의 정확도가 더 낮게 나왔다
> 10 에폭 : 40s 56ms/step - loss: 0.1237 - accuracy: 0.9569 - val_loss: 173.4912 - val_accuracy: 0.5017  
> 10 에폭 : 40s 55ms/step - loss: 0.1524 - accuracy: 0.9485 - val_loss: 25.6186 - val_accuracy: 0.6407 (21.02.01)

## 유선진동 데이터

In [ ]:
pred_name = ['bearing', 'belt', 'misalignment','normal', 'unbalance']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kimm_x,kimm_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

### 일반적인 다층 퍼셉트론(MLP)

In [ ]:
model2 = keras.models.Sequential()
model2.add(keras.layers.Flatten(input_shape=[60, 2]))
model2.add(keras.layers.Dense(300, activation="relu"))
model2.add(keras.layers.Dense(100, activation="relu"))
model2.add(keras.layers.Dense(5, activation="softmax")) 

In [ ]:
model2.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
history2 = model2.fit(X_train,y_train,epochs=30,validation_data=(X_valid,y_valid))

Epoch 1/30
1440/1440 [==============================] - 3s 2ms/step - loss: 2.1606 - accuracy: 0.4907 - val_loss: 1.3745 - val_accuracy: 0.5014
Epoch 2/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3761 - accuracy: 0.4982 - val_loss: 1.3709 - val_accuracy: 0.5014
Epoch 3/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3733 - accuracy: 0.4999 - val_loss: 1.3721 - val_accuracy: 0.5014
Epoch 4/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3722 - accuracy: 0.4998 - val_loss: 1.3707 - val_accuracy: 0.5014
Epoch 5/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3702 - accuracy: 0.5013 - val_loss: 1.3697 - val_accuracy: 0.5014
Epoch 6/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3708 - accuracy: 0.5009 - val_loss: 1.3711 - val_accuracy: 0.5014
Epoch 7/30
1440/1440 [==============================] - 3s 2ms/step - loss: 1.3680 - accuracy: 0.5015 - val_loss: 1.3692 - val_accuracy:

정확도가 50%를 웃도는 결과를 나타냄
> 30 에폭 : 3s 2ms/step - loss: 1.3632 - accuracy: 0.4964 - val_loss: 1.3568 - val_accuracy: 0.5014

### CNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kimm_x,kimm_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(46080, 60, 2, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
from functools import partial

cnn_model2 = keras.models.Sequential([
    keras.layers.Conv2D(64,7,activation="relu",padding="same",input_shape=(60,2,1)),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=5, activation='softmax')
])

In [ ]:
cnn_model2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
history = cnn_model2.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1440/1440 [==============================] - 23s 10ms/step - loss: 1.4028 - accuracy: 0.4969 - val_loss: 1.3660 - val_accuracy: 0.5014
Epoch 2/10
1440/1440 [==============================] - 15s 10ms/step - loss: 1.3725 - accuracy: 0.4983 - val_loss: 1.3935 - val_accuracy: 0.5014
Epoch 3/10
1440/1440 [==============================] - 15s 10ms/step - loss: 1.3710 - accuracy: 0.5003 - val_loss: 1.3687 - val_accuracy: 0.5014
Epoch 4/10
1440/1440 [==============================] - 15s 10ms/step - loss: 1.3672 - accuracy: 0.4998 - val_loss: 1.3724 - val_accuracy: 0.5014
Epoch 5/10
1440/1440 [==============================] - 15s 10ms/step - loss: 1.3581 - accuracy: 0.5014 - val_loss: 1.3605 - val_accuracy: 0.5014
Epoch 6/10
1440/1440 [==============================] - 15s 11ms/step - loss: 1.3581 - accuracy: 0.5009 - val_loss: 1.3572 - val_accuracy: 0.5014
Epoch 7/10
1440/1440 [==============================] - 15s 11ms/step - loss: 1.3518 - accuracy: 0.5016 - val_loss: 1.3506 -

전류 데이터와 달리 유선 진동 데이터는 CNN 을 적용해도 훈련,검증 세트의 정확도가 50%를 웃도는 결과를 나타냄
> 10 에폭 : 15s 11ms/step - loss: 1.2736 - accuracy: 0.4999 - val_loss: 1.1014 - val_accuracy: 0.5346

In [ ]:
X_new = X_train[:10]
y_pred = cnn_model2.predict_classes(X_new)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [ ]:
y_train[:10]

array([3, 1, 3, 0, 4, 3, 3, 3, 0, 3])

### CNN + 배치정규화

In [ ]:
cnn_model2_1 = keras.models.Sequential([
    keras.layers.Conv2D(64,7,padding="same",input_shape=(60,2,1),use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(128,3,padding="same",use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Conv2D(128,3,padding="same",use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(256,3,padding="same",use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Conv2D(256,3,padding="same",use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=5, activation='softmax')
])

In [ ]:
cnn_model2_1.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
cnn_model2_1.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1440/1440 [==============================] - 18s 11ms/step - loss: 1.5930 - accuracy: 0.4689 - val_loss: 1.3677 - val_accuracy: 0.5014
Epoch 2/10
1440/1440 [==============================] - 17s 12ms/step - loss: 1.3811 - accuracy: 0.4980 - val_loss: 1.3622 - val_accuracy: 0.5014
Epoch 3/10
1440/1440 [==============================] - 16s 11ms/step - loss: 1.3850 - accuracy: 0.5001 - val_loss: 1.3632 - val_accuracy: 0.5014
Epoch 4/10
1440/1440 [==============================] - 17s 11ms/step - loss: 1.3733 - accuracy: 0.4997 - val_loss: 1.3611 - val_accuracy: 0.5014
Epoch 5/10
1440/1440 [==============================] - 17s 11ms/step - loss: 1.3794 - accuracy: 0.5014 - val_loss: 1.3701 - val_accuracy: 0.5014
Epoch 6/10
1440/1440 [==============================] - 17s 12ms/step - loss: 1.3715 - accuracy: 0.5009 - val_loss: 1.3654 - val_accuracy: 0.5014
Epoch 7/10
1440/1440 [==============================] - 17s 12ms/step - loss: 1.3682 - accuracy: 0.5014 - val_loss: 1.3583 -

기본 CNN에 배치정규화를 추가해도 이전보다 좋은 성능이 나오지 않음, 오히려 기존보다 미미하게 나쁜 결과를 나타냄
> 10 에폭 : 17s 12ms/step - loss: 1.3545 - accuracy: 0.4987 - val_loss: 1.3512 - val_accuracy: 0.5014

### ResNet-34 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(kimm_x,kimm_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(46080, 60, 2, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
kimm_resnet_model = keras.models.Sequential()
kimm_resnet_model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[60,2,1],padding="same",use_bias=False))
kimm_resnet_model.add(keras.layers.BatchNormalization())
kimm_resnet_model.add(keras.layers.Activation("relu"))
kimm_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    kimm_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
kimm_resnet_model.add(keras.layers.GlobalAvgPool2D())
kimm_resnet_model.add(keras.layers.Flatten())
kimm_resnet_model.add(keras.layers.Dense(5,activation="softmax"))

In [ ]:
kimm_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
kimm_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1440/1440 [==============================] - 78s 49ms/step - loss: 1.3476 - accuracy: 0.5199 - val_loss: 2.2187 - val_accuracy: 0.5486
Epoch 2/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.8356 - accuracy: 0.6544 - val_loss: 1.7420 - val_accuracy: 0.4881
Epoch 3/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.6888 - accuracy: 0.7121 - val_loss: 2.2917 - val_accuracy: 0.4761
Epoch 4/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.7407 - accuracy: 0.7009 - val_loss: 2.5210 - val_accuracy: 0.4584
Epoch 5/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.7434 - accuracy: 0.7005 - val_loss: 1.3602 - val_accuracy: 0.6928
Epoch 6/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.5244 - accuracy: 0.7724 - val_loss: 1.4629 - val_accuracy: 0.6686
Epoch 7/10
1440/1440 [==============================] - 70s 49ms/step - loss: 0.4968 - accuracy: 0.7820 - val_loss: 1.7292 -

CNN 보다 훈련, 검증 세트 둘 다 정확도는 높아졌으나, __검증 세트의 정확도__가 __훈련세트의 정확도__보다 __낮게__ 나타남
> 10 에폭 : 70s 49ms/step - loss: 0.3481 - accuracy: 0.8481 - val_loss: 1.2782 - val_accuracy: 0.6444  
> 10 에폭 : 70s 48ms/step - loss: 0.3564 - accuracy: 0.8385 - val_loss: 1.5503 - val_accuracy: 0.7348 ( 21.02.01)

## 무선진동 데이터

무선진동 데이터는 행이 두 가지의 종류(3000 row, 6000 row)로 존재해서

3000 row인 데이터는 나머지 빈 데이터 3000 row를 0으로 채움.

In [ ]:
vibration_x.shape, vibration_y.shape

((38400, 60, 2), (38400, 60))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vibration_x,vibration_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

### 일반적인 다층 퍼셉트론(MLP)

In [ ]:
model3 = keras.models.Sequential()
model3.add(keras.layers.Flatten(input_shape=[60, 2]))
model3.add(keras.layers.Dense(300, activation="relu"))
model3.add(keras.layers.Dense(100, activation="relu"))
model3.add(keras.layers.Dense(5, activation="softmax")) 

In [ ]:
model3.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
history3 = model3.fit(X_train,y_train,epochs=30,validation_data=(X_valid,y_valid))

Epoch 1/30
720/720 [==============================] - 2s 2ms/step - loss: 2.0473 - accuracy: 0.4786 - val_loss: 1.3984 - val_accuracy: 0.4931
Epoch 2/30
720/720 [==============================] - 1s 2ms/step - loss: 1.3846 - accuracy: 0.4993 - val_loss: 1.3825 - val_accuracy: 0.4931
Epoch 3/30
720/720 [==============================] - 2s 2ms/step - loss: 1.3791 - accuracy: 0.4973 - val_loss: 1.3860 - val_accuracy: 0.4931
Epoch 4/30
720/720 [==============================] - 1s 2ms/step - loss: 1.3666 - accuracy: 0.5050 - val_loss: 1.3820 - val_accuracy: 0.4931
Epoch 5/30
720/720 [==============================] - 2s 2ms/step - loss: 1.3848 - accuracy: 0.4916 - val_loss: 1.3808 - val_accuracy: 0.4931
Epoch 6/30
720/720 [==============================] - 1s 2ms/step - loss: 1.3762 - accuracy: 0.4973 - val_loss: 1.3817 - val_accuracy: 0.4931
Epoch 7/30
720/720 [==============================] - 1s 2ms/step - loss: 1.3717 - accuracy: 0.5009 - val_loss: 1.3795 - val_accuracy: 0.4931
Epoch 

정확도가 50%를 웃돔.

30 에폭. 2s 2ms/step - loss: 1.3499 - accuracy: 0.5069 - val_loss: 1.3578 - val_accuracy: 0.4996

### CNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vibration_x,vibration_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape

(23040, 60, 2, 1)

In [ ]:
# 각 특성 값의 범위를 0~1로 조정
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
from functools import partial

cnn_model3 = keras.models.Sequential([
    keras.layers.Conv2D(64,7,activation="relu",padding="same",input_shape=(60,2,1)),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.Conv2D(128,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(1),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.Conv2D(256,3,activation="relu",padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=5, activation='softmax')
])

In [ ]:
cnn_model3.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [ ]:
history = cnn_model3.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
720/720 [==============================] - 9s 11ms/step - loss: 0.9097 - accuracy: 0.5870 - val_loss: 0.7296 - val_accuracy: 0.6355
Epoch 2/10
720/720 [==============================] - 7s 10ms/step - loss: 0.8177 - accuracy: 0.6315 - val_loss: 0.7059 - val_accuracy: 0.6354
Epoch 3/10
720/720 [==============================] - 7s 10ms/step - loss: 0.7080 - accuracy: 0.6330 - val_loss: 0.7099 - val_accuracy: 0.6397
Epoch 4/10
720/720 [==============================] - 8s 10ms/step - loss: 0.6955 - accuracy: 0.6418 - val_loss: 0.7011 - val_accuracy: 0.6354
Epoch 5/10
720/720 [==============================] - 7s 10ms/step - loss: 0.7259 - accuracy: 0.6292 - val_loss: 0.7667 - val_accuracy: 0.6237
Epoch 6/10
720/720 [==============================] - 8s 10ms/step - loss: 0.7267 - accuracy: 0.6364 - val_loss: 0.6976 - val_accuracy: 0.6546
Epoch 7/10
720/720 [==============================] - 8s 10ms/step - loss: 0.7135 - accuracy: 0.6473 - val_loss: 0.6820 - val_accuracy: 0.6355

이전보다 좋아졌지만 그럼에도 정확도가 70%를 넘지 못하는 결과를 보임
> 10 에폭 : 8s 11ms/step - loss: 0.6141 - accuracy: 0.6907 - val_loss: 0.6035 - val_accuracy: 0.6785  

데이터를 수정해서 데이터 패턴을 판단하기 어려워져 정확도가 낮아진 것 같으나 더 좋은 방향임
> 8s 11ms/step - loss: 0.8233 - accuracy: 0.6377 - val_loss: 0.6749 - val_accuracy: 0.6566 ( 21.02.01)

### Resnet34

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vibration_x,vibration_y[:,0],train_size = 0.6, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_train[:,:,1:] = X_train[:,:,1:]/100
X_valid[:,:,1:] = X_valid[:,:,1:]/100
X_test[:,:,1:] = X_test[:,:,1:]/100

In [ ]:
# 잔차유닛
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3,strides=1,padding="same",use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding="same",use_bias=False),
                keras.layers.BatchNormalization()]
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
vibration_resnet_model = keras.models.Sequential()
vibration_resnet_model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[60,2,1],padding="same",use_bias=False))
vibration_resnet_model.add(keras.layers.BatchNormalization())
vibration_resnet_model.add(keras.layers.Activation("relu"))
vibration_resnet_model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="same"))
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
    strides = 1 if filters == prev_filters else 2
    vibration_resnet_model.add(ResidualUnit(filters,strides=strides))
    prev_filters=filters
vibration_resnet_model.add(keras.layers.GlobalAvgPool2D())
vibration_resnet_model.add(keras.layers.Flatten())
vibration_resnet_model.add(keras.layers.Dense(5,activation="softmax"))
vibration_resnet_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
vibration_resnet_model.fit(X_train,y_train,epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
720/720 [==============================] - 42s 49ms/step - loss: 0.8951 - accuracy: 0.6152 - val_loss: 0.6757 - val_accuracy: 0.6582
Epoch 2/10
720/720 [==============================] - 35s 49ms/step - loss: 0.6789 - accuracy: 0.6663 - val_loss: 0.6402 - val_accuracy: 0.6961
Epoch 3/10
720/720 [==============================] - 35s 49ms/step - loss: 0.6526 - accuracy: 0.6672 - val_loss: 0.7519 - val_accuracy: 0.6849
Epoch 4/10
720/720 [==============================] - 35s 49ms/step - loss: 0.6701 - accuracy: 0.6668 - val_loss: 0.6186 - val_accuracy: 0.6887
Epoch 5/10
720/720 [==============================] - 35s 49ms/step - loss: 0.6639 - accuracy: 0.6704 - val_loss: 0.5882 - val_accuracy: 0.6966
Epoch 6/10
720/720 [==============================] - 35s 49ms/step - loss: 0.6221 - accuracy: 0.6907 - val_loss: 0.5793 - val_accuracy: 0.7206
Epoch 7/10
720/720 [==============================] - 35s 49ms/step - loss: 0.5858 - accuracy: 0.7067 - val_loss: 0.5748 - val_accuracy:

이전과 유사하지만 미미하게 성능이 좋아졌다.
> 10 에폭 : 35s 49ms/step - loss: 0.6046 - accuracy: 0.6939 - val_loss: 0.6169 - val_accuracy: 0.6625

데이터 수정 이후 정확도가 70%를 넘어 이전보다 성능이 좋아진 결과를 나타냄
> 10 에폭 : 35s 49ms/step - loss: 0.5614 - accuracy: 0.7174 - val_loss: 0.5437 - val_accuracy: 0.7150

# 테스트 세트

In [ ]:
TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/testSet'

In [ ]:
import csv

In [ ]:
def read_test_dir(path):
    if os.path.isdir(path):
        dir_list = os.listdir(path)
    dir_list = sorted(dir_list)
    
    result_f = os.path.join(path,dir_list[-1])

    tmp_dir_list = []
    with open(result_f,'r') as rf:
        tt = csv.reader(rf)
        for i in tt:
            tmp_dir_list.append(i)
    
    # print(len(tmp_dir_list))

    tmp_dir_list = tmp_dir_list[1:]
    test_dir_list = []
    category = []
    kw = []
    for t in tmp_dir_list:
        num = t[0]
        if len(num)<2:
            num = '00'+num
        elif len(num)<3:
            num = '0'+num
        f_name = num+'.csv'
        p = os.path.join(path,f_name)
        test_dir_list.append(p)
        category.append(t[1])
        kw.append(t[2])

    return test_dir_list, category, kw

In [ ]:
def test_predict():
    t,c,k = read_test_dir(TEST_PATH)
    y = {}
    X = None
    for i in range(len(t)):
        f_name = os.path.basename(t[i])
        if f_name == '142.csv':
            print('{} pass : Current라고 되어있으나 3 col이 아닌 1col data, 주최측은 Current가 맞다고 답변'.format(f_name))
            pass
        elif c[i] == 'Current':
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100
            
            r1 = cnn_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            r2 = current_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n cnn : {}\n resnet : {}'.format(f_name,c[i],r1,r2))
            y[f_name] = [r1,r2]
        elif c[i] == 'Kimm':
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100

            r1 = kimm_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n resnet : {}'.format(f_name,c[i],r1))
            y[f_name] = [r1]
        else: # Vibration
            X = pd.read_csv(t[i])
            X.drop('Seq',axis=1,inplace=True)
            X['Motor'] = float(k[i])/100

            r1 = vibration_resnet_model.predict_classes(X.values[np.newaxis,...,np.newaxis])
            print('{}, {}\n resnet : {}'.format(f_name,c[i],r1))
            y[f_name] = [r1]
    return y

In [ ]:
test_y_dict = test_predict()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


001.csv, Kimm
 resnet : [3]
002.csv, Kimm
 resnet : [3]
003.csv, Vibration
 resnet : [4]
004.csv, Current
 cnn : [1]
 resnet : [3]
005.csv, Vibration
 resnet : [4]
006.csv, Current
 cnn : [3]
 resnet : [3]
007.csv, Kimm
 resnet : [3]
008.csv, Vibration
 resnet : [1]
009.csv, Kimm
 resnet : [3]
010.csv, Vibration
 resnet : [4]
011.csv, Kimm
 resnet : [3]
012.csv, Vibration
 resnet : [1]
013.csv, Current
 cnn : [3]
 resnet : [3]
014.csv, Vibration
 resnet : [2]
015.csv, Vibration
 resnet : [4]
016.csv, Kimm
 resnet : [4]
017.csv, Current
 cnn : [2]
 resnet : [3]
018.csv, Kimm
 resnet : [3]
019.csv, Vibration
 resnet : [1]
020.csv, Kimm
 resnet : [3]
021.csv, Vibration
 resnet : [1]
022.csv, Current
 cnn : [0]
 resnet : [3]
023.csv, Kimm
 resnet : [3]
024.csv, Kimm
 resnet : [0]
025.csv, Vibration
 resnet : [4]
026.csv, Kimm
 resnet : [0]
027.csv, Vibration
 resnet : [1]
028.csv, Vibration
 resnet : [2]
029.csv, Kimm
 resnet : [3]
030.csv, Vibration
 resnet : [4]
031.csv, Current
 cnn : [